In [15]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import numpy as np

In [16]:
train_data = pd.read_csv("labeled_file.txt", sep="\t", header=None)
test_data = pd.read_csv("unlabeled_file.txt", sep="\t", header=None)

In [17]:
layers_dim = [21, 50, 128]

In [18]:
class CNNNet(torch.nn.Module):
    def __init__(self):
        super(CNNNet, self).__init__()
        
        self.conv1 = torch.nn.Conv1d(1, 8, kernel_size=3, padding=1, stride=1)
        self.pool1 = torch.nn.MaxPool1d(kernel_size=2, stride=2, padding=0)
        self.drop = torch.nn.Dropout(0.5)
        self.fc1 = torch.nn.Linear(80, 128)
        self.fc2 = torch.nn.Linear(128, 128)
        
    def forward(self, x):
        x = F.relu(self.conv1(x))
        
        x = self.pool1(x)
        
        x = x.view(-1, 80)
        
        x = F.relu(self.fc1(x))
        x = self.drop(x)
        x = (self.fc2(x))
        
        return x
    
CNNnet = CNNNet()

In [19]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.fc1 = torch.nn.Linear(layers_dim[0], layers_dim[1])
        self.relu1 = torch.nn.ReLU()
        self.drop = torch.nn.Dropout(0.5)
        self.fc2 = torch.nn.Linear(layers_dim[1], layers_dim[2])
        
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.drop(x)
        x = self.fc2(x)
        
        return x
    
net = Net()

In [20]:
#USE THIS ONLY FOR LOADING THE MODEL

netLOADER = torch.nn.Sequential(
    torch.nn.Linear(layers_dim[0], layers_dim[1]),
    torch.nn.ReLU(),
    torch.nn.Dropout(0.5),
    torch.nn.Linear(layers_dim[1], layers_dim[2]),
)

In [ ]:
CNNNetLoader = torch.nn.Sequential(
    torch.nn.Conv1d(1, 8, kernel_size=3, padding=1, stride=1)
    torch.nn.MaxPool1d(kernel_size=2, stride=2, padding=0)
    
)

In [21]:
train_x = train_data.iloc[:, 2:-1]
test_x = test_data.iloc[:, 2:-1]
train_y = train_data.iloc[:, -1]
train_y.value_counts()

1      1382
0       462
7       182
12      170
2       146
5       125
11      101
15       99
10       98
18       83
9        81
8        80
30       78
13       71
25       63
27       61
3        61
4        58
49       56
16       48
34       45
38       45
45       43
19       41
29       37
41       32
39       31
24       29
53       28
64       27
       ... 
115       3
54        3
123       3
51        3
93        3
113       3
127       2
108       2
118       2
111       2
95        2
91        2
79        2
55        2
47        2
35        2
126       2
122       2
98        2
112       2
94        2
86        2
66        2
117       2
109       2
81        2
77        2
120       2
116       2
125       2
Name: 23, Length: 128, dtype: int64

In [22]:
print(train_x.shape, test_x.shape, train_y.shape)

(4636, 21) (19, 21) (4636,)


In [23]:
print(net.parameters)

<bound method Module.parameters of Net(
  (fc1): Linear(in_features=21, out_features=50, bias=True)
  (relu1): ReLU()
  (drop): Dropout(p=0.5)
  (fc2): Linear(in_features=50, out_features=128, bias=True)
)>


In [24]:
print(CNNnet.parameters)

<bound method Module.parameters of CNNNet(
  (conv1): Conv1d(1, 8, kernel_size=(3,), stride=(1,), padding=(1,))
  (pool1): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (drop): Dropout(p=0.5)
  (fc1): Linear(in_features=80, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=128, bias=True)
)>


In [25]:
train_x = np.array(train_x.values, dtype=np.float32)
train_y = np.array(train_y.values, dtype=np.long)
test_x = np.array(test_x.values, dtype=np.float32)
train_y = np.reshape(train_y, (4636, ))
train_x = torch.from_numpy(train_x)
test_x = torch.from_numpy(test_x)
train_y = torch.from_numpy(train_y)

In [26]:
print(train_x.shape, test_x.shape, train_y.shape)

torch.Size([4636, 21]) torch.Size([19, 21]) torch.Size([4636])


In [27]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=0.001)
num_epochs = 10000
batch_size = 32

In [14]:
for epoch in range(num_epochs):  # number of epochs
    outputs = net(train_x)
    loss = criterion(outputs, train_y)
    
    #backward
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    print ('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

Epoch [1/10000], Loss: 173.1446
Epoch [2/10000], Loss: 165.6866
Epoch [3/10000], Loss: 157.3281
Epoch [4/10000], Loss: 151.3768
Epoch [5/10000], Loss: 145.7855
Epoch [6/10000], Loss: 137.0488
Epoch [7/10000], Loss: 131.1355
Epoch [8/10000], Loss: 128.3500
Epoch [9/10000], Loss: 119.3863
Epoch [10/10000], Loss: 115.3009
Epoch [11/10000], Loss: 108.8156
Epoch [12/10000], Loss: 105.7302
Epoch [13/10000], Loss: 99.5586
Epoch [14/10000], Loss: 100.4561
Epoch [15/10000], Loss: 94.6479
Epoch [16/10000], Loss: 87.3074
Epoch [17/10000], Loss: 85.3248
Epoch [18/10000], Loss: 81.1983
Epoch [19/10000], Loss: 76.9189
Epoch [20/10000], Loss: 78.3245
Epoch [21/10000], Loss: 76.2543
Epoch [22/10000], Loss: 70.7140
Epoch [23/10000], Loss: 65.0138
Epoch [24/10000], Loss: 64.3251
Epoch [25/10000], Loss: 61.4297
Epoch [26/10000], Loss: 59.4760
Epoch [27/10000], Loss: 57.7612
Epoch [28/10000], Loss: 55.6299
Epoch [29/10000], Loss: 51.5329
Epoch [30/10000], Loss: 49.7441
Epoch [31/10000], Loss: 49.5879
Epoc

Epoch [260/10000], Loss: 4.3272
Epoch [261/10000], Loss: 4.3350
Epoch [262/10000], Loss: 4.3236
Epoch [263/10000], Loss: 4.2857
Epoch [264/10000], Loss: 4.3191
Epoch [265/10000], Loss: 4.3854
Epoch [266/10000], Loss: 4.3795
Epoch [267/10000], Loss: 4.3467
Epoch [268/10000], Loss: 4.3352
Epoch [269/10000], Loss: 4.3029
Epoch [270/10000], Loss: 4.3516
Epoch [271/10000], Loss: 4.2870
Epoch [272/10000], Loss: 4.2976
Epoch [273/10000], Loss: 4.2661
Epoch [274/10000], Loss: 4.2950
Epoch [275/10000], Loss: 4.1987
Epoch [276/10000], Loss: 4.3104
Epoch [277/10000], Loss: 4.2777
Epoch [278/10000], Loss: 4.3066
Epoch [279/10000], Loss: 4.3040
Epoch [280/10000], Loss: 4.2948
Epoch [281/10000], Loss: 4.2583
Epoch [282/10000], Loss: 4.2818
Epoch [283/10000], Loss: 4.2241
Epoch [284/10000], Loss: 4.2843
Epoch [285/10000], Loss: 4.2687
Epoch [286/10000], Loss: 4.2269
Epoch [287/10000], Loss: 4.2473
Epoch [288/10000], Loss: 4.2467
Epoch [289/10000], Loss: 4.1845
Epoch [290/10000], Loss: 4.1984
Epoch [2

Epoch [519/10000], Loss: 3.7441
Epoch [520/10000], Loss: 3.6832
Epoch [521/10000], Loss: 3.6928
Epoch [522/10000], Loss: 3.7203
Epoch [523/10000], Loss: 3.7025
Epoch [524/10000], Loss: 3.7227
Epoch [525/10000], Loss: 3.7013
Epoch [526/10000], Loss: 3.7098
Epoch [527/10000], Loss: 3.6924
Epoch [528/10000], Loss: 3.7433
Epoch [529/10000], Loss: 3.7482
Epoch [530/10000], Loss: 3.7198
Epoch [531/10000], Loss: 3.6593
Epoch [532/10000], Loss: 3.7117
Epoch [533/10000], Loss: 3.6866
Epoch [534/10000], Loss: 3.6742
Epoch [535/10000], Loss: 3.6960
Epoch [536/10000], Loss: 3.6784
Epoch [537/10000], Loss: 3.6744
Epoch [538/10000], Loss: 3.6756
Epoch [539/10000], Loss: 3.7203
Epoch [540/10000], Loss: 3.6643
Epoch [541/10000], Loss: 3.6914
Epoch [542/10000], Loss: 3.6615
Epoch [543/10000], Loss: 3.6870
Epoch [544/10000], Loss: 3.6728
Epoch [545/10000], Loss: 3.6544
Epoch [546/10000], Loss: 3.6895
Epoch [547/10000], Loss: 3.6498
Epoch [548/10000], Loss: 3.6708
Epoch [549/10000], Loss: 3.6604
Epoch [5

Epoch [779/10000], Loss: 3.4097
Epoch [780/10000], Loss: 3.3976
Epoch [781/10000], Loss: 3.4127
Epoch [782/10000], Loss: 3.3826
Epoch [783/10000], Loss: 3.3820
Epoch [784/10000], Loss: 3.3787
Epoch [785/10000], Loss: 3.3841
Epoch [786/10000], Loss: 3.4058
Epoch [787/10000], Loss: 3.3680
Epoch [788/10000], Loss: 3.3810
Epoch [789/10000], Loss: 3.3904
Epoch [790/10000], Loss: 3.4007
Epoch [791/10000], Loss: 3.4129
Epoch [792/10000], Loss: 3.3855
Epoch [793/10000], Loss: 3.3889
Epoch [794/10000], Loss: 3.4061
Epoch [795/10000], Loss: 3.3760
Epoch [796/10000], Loss: 3.3863
Epoch [797/10000], Loss: 3.3953
Epoch [798/10000], Loss: 3.3886
Epoch [799/10000], Loss: 3.3742
Epoch [800/10000], Loss: 3.3878
Epoch [801/10000], Loss: 3.3869
Epoch [802/10000], Loss: 3.3760
Epoch [803/10000], Loss: 3.4150
Epoch [804/10000], Loss: 3.3891
Epoch [805/10000], Loss: 3.3972
Epoch [806/10000], Loss: 3.3915
Epoch [807/10000], Loss: 3.3942
Epoch [808/10000], Loss: 3.3821
Epoch [809/10000], Loss: 3.3744
Epoch [8

Epoch [1034/10000], Loss: 3.2174
Epoch [1035/10000], Loss: 3.2243
Epoch [1036/10000], Loss: 3.2435
Epoch [1037/10000], Loss: 3.2272
Epoch [1038/10000], Loss: 3.2401
Epoch [1039/10000], Loss: 3.2190
Epoch [1040/10000], Loss: 3.2285
Epoch [1041/10000], Loss: 3.2238
Epoch [1042/10000], Loss: 3.2434
Epoch [1043/10000], Loss: 3.2204
Epoch [1044/10000], Loss: 3.2278
Epoch [1045/10000], Loss: 3.2201
Epoch [1046/10000], Loss: 3.2466
Epoch [1047/10000], Loss: 3.2171
Epoch [1048/10000], Loss: 3.2179
Epoch [1049/10000], Loss: 3.2169
Epoch [1050/10000], Loss: 3.2268
Epoch [1051/10000], Loss: 3.2233
Epoch [1052/10000], Loss: 3.2186
Epoch [1053/10000], Loss: 3.2165
Epoch [1054/10000], Loss: 3.2212
Epoch [1055/10000], Loss: 3.2134
Epoch [1056/10000], Loss: 3.2133
Epoch [1057/10000], Loss: 3.2387
Epoch [1058/10000], Loss: 3.2290
Epoch [1059/10000], Loss: 3.2124
Epoch [1060/10000], Loss: 3.2085
Epoch [1061/10000], Loss: 3.2271
Epoch [1062/10000], Loss: 3.2027
Epoch [1063/10000], Loss: 3.2229
Epoch [106

...

Epoch [8797/10000], Loss: 2.0128
Epoch [8798/10000], Loss: 2.0166
Epoch [8799/10000], Loss: 1.9892
Epoch [8800/10000], Loss: 1.9870
Epoch [8801/10000], Loss: 1.9896
Epoch [8802/10000], Loss: 1.9999
Epoch [8803/10000], Loss: 2.0014
Epoch [8804/10000], Loss: 1.9894
Epoch [8805/10000], Loss: 2.0041
Epoch [8806/10000], Loss: 2.0047
Epoch [8807/10000], Loss: 1.9919
Epoch [8808/10000], Loss: 2.0143
Epoch [8809/10000], Loss: 2.0211
Epoch [8810/10000], Loss: 2.0066
Epoch [8811/10000], Loss: 2.0171
Epoch [8812/10000], Loss: 2.0000
Epoch [8813/10000], Loss: 2.0046
Epoch [8814/10000], Loss: 1.9815
Epoch [8815/10000], Loss: 1.9981
Epoch [8816/10000], Loss: 2.0285
Epoch [8817/10000], Loss: 1.9855
Epoch [8818/10000], Loss: 1.9784
Epoch [8819/10000], Loss: 1.9895
Epoch [8820/10000], Loss: 1.9962
Epoch [8821/10000], Loss: 1.9881
Epoch [8822/10000], Loss: 1.9732
Epoch [8823/10000], Loss: 1.9939
Epoch [8824/10000], Loss: 1.9941
Epoch [8825/10000], Loss: 1.9960
Epoch [8826/10000], Loss: 2.0068
Epoch [882

Epoch [9049/10000], Loss: 1.9863
Epoch [9050/10000], Loss: 1.9813
Epoch [9051/10000], Loss: 1.9984
Epoch [9052/10000], Loss: 1.9924
Epoch [9053/10000], Loss: 2.0028
Epoch [9054/10000], Loss: 1.9776
Epoch [9055/10000], Loss: 1.9907
Epoch [9056/10000], Loss: 1.9823
Epoch [9057/10000], Loss: 1.9801
Epoch [9058/10000], Loss: 1.9946
Epoch [9059/10000], Loss: 1.9851
Epoch [9060/10000], Loss: 1.9921
Epoch [9061/10000], Loss: 1.9915
Epoch [9062/10000], Loss: 1.9975
Epoch [9063/10000], Loss: 1.9744
Epoch [9064/10000], Loss: 1.9751
Epoch [9065/10000], Loss: 1.9879
Epoch [9066/10000], Loss: 1.9821
Epoch [9067/10000], Loss: 1.9870
Epoch [9068/10000], Loss: 1.9798
Epoch [9069/10000], Loss: 1.9834
Epoch [9070/10000], Loss: 1.9923
Epoch [9071/10000], Loss: 1.9821
Epoch [9072/10000], Loss: 1.9826
Epoch [9073/10000], Loss: 1.9960
Epoch [9074/10000], Loss: 1.9968
Epoch [9075/10000], Loss: 1.9676
Epoch [9076/10000], Loss: 1.9723
Epoch [9077/10000], Loss: 1.9746
Epoch [9078/10000], Loss: 1.9754
Epoch [907

Epoch [9301/10000], Loss: 1.9206
Epoch [9302/10000], Loss: 1.9092
Epoch [9303/10000], Loss: 1.9139
Epoch [9304/10000], Loss: 1.8992
Epoch [9305/10000], Loss: 1.9286
Epoch [9306/10000], Loss: 1.9344
Epoch [9307/10000], Loss: 1.9210
Epoch [9308/10000], Loss: 1.9019
Epoch [9309/10000], Loss: 1.9151
Epoch [9310/10000], Loss: 1.9149
Epoch [9311/10000], Loss: 1.9196
Epoch [9312/10000], Loss: 1.9110
Epoch [9313/10000], Loss: 1.9102
Epoch [9314/10000], Loss: 1.9208
Epoch [9315/10000], Loss: 1.9185
Epoch [9316/10000], Loss: 1.9147
Epoch [9317/10000], Loss: 1.9259
Epoch [9318/10000], Loss: 1.9108
Epoch [9319/10000], Loss: 1.9043
Epoch [9320/10000], Loss: 1.9100
Epoch [9321/10000], Loss: 1.9251
Epoch [9322/10000], Loss: 1.9116
Epoch [9323/10000], Loss: 1.9150
Epoch [9324/10000], Loss: 1.9257
Epoch [9325/10000], Loss: 1.8965
Epoch [9326/10000], Loss: 1.9168
Epoch [9327/10000], Loss: 1.9099
Epoch [9328/10000], Loss: 1.9062
Epoch [9329/10000], Loss: 1.9240
Epoch [9330/10000], Loss: 1.9242
Epoch [933

Epoch [9553/10000], Loss: 1.8869
Epoch [9554/10000], Loss: 1.9032
Epoch [9555/10000], Loss: 1.8821
Epoch [9556/10000], Loss: 1.9041
Epoch [9557/10000], Loss: 1.9021
Epoch [9558/10000], Loss: 1.9020
Epoch [9559/10000], Loss: 1.9046
Epoch [9560/10000], Loss: 1.8895
Epoch [9561/10000], Loss: 1.8848
Epoch [9562/10000], Loss: 1.8833
Epoch [9563/10000], Loss: 1.8742
Epoch [9564/10000], Loss: 1.8760
Epoch [9565/10000], Loss: 1.8770
Epoch [9566/10000], Loss: 1.8969
Epoch [9567/10000], Loss: 1.8725
Epoch [9568/10000], Loss: 1.8794
Epoch [9569/10000], Loss: 1.8920
Epoch [9570/10000], Loss: 1.9072
Epoch [9571/10000], Loss: 1.8997
Epoch [9572/10000], Loss: 1.8929
Epoch [9573/10000], Loss: 1.8853
Epoch [9574/10000], Loss: 1.8896
Epoch [9575/10000], Loss: 1.8970
Epoch [9576/10000], Loss: 1.8943
Epoch [9577/10000], Loss: 1.8899
Epoch [9578/10000], Loss: 1.8856
Epoch [9579/10000], Loss: 1.8782
Epoch [9580/10000], Loss: 1.8802
Epoch [9581/10000], Loss: 1.8862
Epoch [9582/10000], Loss: 1.8793
Epoch [958

Epoch [9804/10000], Loss: 1.8570
Epoch [9805/10000], Loss: 1.8728
Epoch [9806/10000], Loss: 1.8790
Epoch [9807/10000], Loss: 1.8863
Epoch [9808/10000], Loss: 1.8632
Epoch [9809/10000], Loss: 1.8669
Epoch [9810/10000], Loss: 1.8717
Epoch [9811/10000], Loss: 1.8735
Epoch [9812/10000], Loss: 1.8489
Epoch [9813/10000], Loss: 1.8654
Epoch [9814/10000], Loss: 1.8545
Epoch [9815/10000], Loss: 1.8560
Epoch [9816/10000], Loss: 1.8618
Epoch [9817/10000], Loss: 1.8626
Epoch [9818/10000], Loss: 1.8712
Epoch [9819/10000], Loss: 1.8699
Epoch [9820/10000], Loss: 1.8780
Epoch [9821/10000], Loss: 1.8745
Epoch [9822/10000], Loss: 1.8567
Epoch [9823/10000], Loss: 1.8540
Epoch [9824/10000], Loss: 1.8723
Epoch [9825/10000], Loss: 1.8594
Epoch [9826/10000], Loss: 1.8779
Epoch [9827/10000], Loss: 1.8688
Epoch [9828/10000], Loss: 1.8630
Epoch [9829/10000], Loss: 1.8520
Epoch [9830/10000], Loss: 1.8552
Epoch [9831/10000], Loss: 1.8537
Epoch [9832/10000], Loss: 1.8698
Epoch [9833/10000], Loss: 1.8582
Epoch [983

In [ ]:
num_batches = train_x.shape[0]//batch_size
for epoch in range(num_epochs):  # number of epochs
    for batch_no in range(num_batches):
        outputs = net(train_x[batch_no*batch_size:(batch_size*(batch_no+1))])
        loss = criterion(outputs, train_y[batch_no*batch_size:(batch_size*(batch_no+1))])
    
        #backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print ('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

In [ ]:
#outputs = net(train_x)

In [ ]:
#_, predicted = torch.max(outputs, 1)

In [ ]:
#print(predicted)

In [ ]:
#correct = (predicted == train_y).sum().item()

In [ ]:
#print("Accuracy = {}%".format(100 *(float(correct/train_data.shape[0]))))

In [42]:
def predict_accuracy(network, val_x, val_y):
    outputs = network(val_x)
    _, predicted = torch.max(outputs, 1)
    correct = (predicted == val_y).sum().item()
    print("Accuracy = {}%".format(100 *(float(correct/val_x.shape[0]))))

In [43]:
predict_accuracy(CNNnet, train_x, train_y)

Accuracy = 51.9197584124245%


In [32]:
def generate_labels(test_x):
    outputs = net(test_x)
    _, predictions = torch.max(outputs, 1)
    return predictions

In [13]:
!ls

CNN Model.ipynb				  trained_model_10000_epochs.pt
labeled_file.txt			  trained_model_CNN_10000_epochs_1.pt
NN model (includes validation set).ipynb  trained_model_CNN_10000_epochs_2.pt
NN Model.ipynb				  unlabeled_file.txt
preprocessed_BCLL.txt


In [18]:
torch.save(net, "trained_model_CNN_10000_epochs_2.pt")

/home/saruu/anaconda3/envs/GPU_ENV_1_TORCH/lib/python3.5/site-packages/torch/serialization.py:193: UserWarning: Couldn't retrieve source code for container of type Net. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [40]:
CNNnet = torch.load("trained_model_CNN_10000_epochs_2.pt")

In [41]:
print(CNNnet)

Net(
  (fc1): Linear(in_features=21, out_features=50, bias=True)
  (relu1): ReLU()
  (drop): Dropout(p=0.5)
  (fc2): Linear(in_features=50, out_features=128, bias=True)
)


In [33]:
test_labels = generate_labels(test_x)
print(test_labels)

tensor([  51,   13,   12,   79,   89,   13,   13,   88,  119,  119,
          79,  123,   79,  119,   51,   79,   68,  116,   79])


In [34]:
test_data[[23]] = test_labels
test_data

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
0,35955_at,CECR7,295.5,392.5,265.5,338.5,461.9,225.3,671.4,364.6,...,189.3,274.3,206.3,494.7,723.0,457.1,573.4,152.3,282.5,51
1,36773_f_at,LOC100293977,2359.3,1782.2,1158.9,2534.7,1091.8,2428.0,2298.5,1792.8,...,1010.5,1972.5,872.9,1603.4,913.6,1769.9,796.1,925.5,1668.8,13
2,37155_at,CNTNAP2,128.9,377.9,30.8,5.7,21.3,148.1,40.2,78.9,...,38.9,98.4,60.5,366.3,38.5,171.8,66.2,57.2,9.3,12
3,32766_at,XRCC6,1529.3,1413.7,1731.0,1363.2,1035.4,1545.3,1659.3,1243.0,...,1520.0,1515.7,1447.6,1358.1,790.0,1442.8,786.5,1227.9,1371.9,79
4,33439_at,SIK1,209.0,17.3,245.9,147.3,88.6,31.4,112.8,23.3,...,64.9,167.1,110.7,23.2,90.6,73.5,208.0,81.5,837.8,89
5,41106_at,KCNN4,555.8,886.7,564.5,668.1,712.0,642.9,337.7,781.1,...,352.1,825.0,639.7,428.8,507.4,1110.7,501.2,694.6,426.0,13
6,31975_at,MT4,111.6,156.5,129.5,116.5,121.5,116.1,71.8,126.3,...,104.5,142.2,133.0,117.8,250.7,102.9,203.5,143.5,157.1,13
7,33274_f_at,CYAT1,13901.9,633.0,9728.8,442.5,4268.7,8484.8,1871.8,9704.9,...,475.3,9905.0,6707.7,434.8,3426.2,202.9,6161.2,2410.3,515.1,88
8,39916_r_at,RPS15,4191.4,2819.6,2824.4,2572.5,2010.2,3440.0,2929.9,2543.4,...,2753.9,2817.4,2720.8,3242.1,1981.6,3950.2,2362.4,2615.2,2748.8,119
9,32432_f_at,RPL15,6688.6,3784.0,3510.2,3383.4,3046.4,4373.9,3684.2,4019.8,...,4302.3,3678.7,2708.7,3545.6,2746.1,5180.2,2124.9,3879.9,3414.8,119


In [36]:
test_data.to_csv("Labelled_prev_unlabelBCLL_CNN_M1.txt", sep=" ", header=False, index=False)